# 2 dolgot kell megoldanom és kész: a fromkeyses dolog nélkül a duplikálás kiszűrését, és a sk esetében a dupla spaceket

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import random

import gkeepapi
keep = gkeepapi.Keep()
keep.login('y.kornel', open('pass').read())

True

##### Ahhoz, hogy a gkeepapi be tudjon jelentkezni a fiókodba, ki kell kapcsolni a kétlépcsős bejelentkezést! A bejelentkezéshez a jelszót a pass nevű txt-ben tárolom.

## Ide másolj be egy recept linket a nosalty.hu-ról vagy a streetckitchen.hu-ról:

In [2]:
url1 = 'https://streetkitchen.hu/hust-hussal/konfitalt-kacsacomb-afonyas-vorosboros-chutney-val-es-sult-paszternakkal/'
url2 = 'https://www.nosalty.hu/recept/vorosboros-paradicsomos-marhaszelet'

## Streetkitchen esetében futtasd le ezt:

In [3]:
def sync_streetkitchen(url1):
    page = requests.get(url1)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    hozzavalok = soup.find_all('dd')

    hozzavalok_lista_lista = []
    for i in hozzavalok:
        hozzavalok_lista_lista.append([i.get_text().strip()])
    
    hozzavalok_lista = []
    for sublist in hozzavalok_lista_lista:
        for item in sublist:
            if item not in hozzavalok_lista:
                hozzavalok_lista.append(item)
        
    recept_neve = soup.find('h1').get_text()

    glist = keep.createList(recept_neve , [])

    for i in hozzavalok_lista:
        glist.add(i, False, gkeepapi.node.NewListItemPlacementValue.Bottom)    

    glist.pinned = True
    glist.color = gkeepapi.node.ColorValue.Yellow
    keep.sync()
    
    instrukciok = soup.find('div', {'class' : "article-main-content"}).find_all('p')

    instrukcio_lista = []

    for i in instrukciok:
        instrukcio_lista.append(i.get_text().replace('\xa0', ''))

    instrukcio_lista_uj = ' '.join(instrukcio_lista).strip().replace("kb.", "kb").split('. ')

    glist = keep.createList(recept_neve , [])

    for i in instrukcio_lista_uj:
        glist.add(i, False, gkeepapi.node.NewListItemPlacementValue.Bottom)

    recept_neve = soup.find('h1').get_text()

    glist.pinned = True
    glist.color = gkeepapi.node.ColorValue.Yellow
    keep.sync()

sync_streetkitchen(url1)

## Nosalty esetében futtasd le ezt:

In [4]:
def sync_nosalty(url2):
    page = requests.get(url2)
    soup = BeautifulSoup(page.content, 'html.parser')

    hozzavalok = soup.find('div', {'class' : "column-block recept-hozzavalok"}).find_all('li')

    hozzavalok_lista = []

    for i in hozzavalok:
        hozzavalok_lista.append(i.get_text())

    recept_neve = soup.find('h1').get_text()

    glist = keep.createList(recept_neve , [])

    for i in hozzavalok_lista:
        glist.add(i, False, gkeepapi.node.NewListItemPlacementValue.Bottom)
    
    glist.pinned = True
    glist.color = gkeepapi.node.ColorValue.Green
    keep.sync()

    instrukciok = soup.find('div', {'class' : "recept-elkeszites"}).find_all('li')
    recept_neve = soup.find('h1').get_text()

    instrukcio_lista = []

    for i in instrukciok:
        instrukcio_lista.append(i.get_text())

    instrukcio_input = ' '.join(instrukcio_lista).strip().replace("kb.", "kb").split('. ')

    glist = keep.createList(recept_neve, [])

    for i in instrukcio_input:
        glist.add(i, False, gkeepapi.node.NewListItemPlacementValue.Bottom)  

    glist.pinned = True
    glist.color = gkeepapi.node.ColorValue.Green
    keep.sync()

sync_nosalty(url2)

### Kiegészítő funkció: 1 random recept kiválasztása a streetkitchen.hu-ról, és áttöltés a keep-be

In [5]:
keresoszo = "sertés"

kereso_url = "https://streetkitchen.hu/?s={}&cat=".format(keresoszo)

kereso_url

page = requests.get(kereso_url)
soup = BeautifulSoup(page.content, 'html.parser')

recept_elements = soup.find_all("a", {"class": "entry-image-link"})

for recept_element in random.sample(recept_elements, 1):
    sync_streetkitchen(recept_element["href"])